In [ ]:
%load_ext autoreload
%autoreload 2

# Multi images transformation to kx-ky space


In [ ]:
import xrheed
from xrheed.io import load_data

import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from pathlib import Path

from xrheed.plotting.overview import plot_images
from xrheed.preparation.filters import high_pass_filter

In [ ]:
image_dir = Path("example_data")
image_paths = image_dir.glob("Si_111_r3Ag_112_phi_*.raw")

beta = 2.85

# Define alphas for each image (include manipulator misalignment)
alpha_vect = [0.0, 14.6, 29.9]

# Define center position for each image
center_xs = [-1.0, -1.0, -0.6]
center_ys = [-0.5, -1.1, -0.5]

rheed_images = []


for i, image_path in enumerate(image_paths):
    rheed_image = load_data(image_path, plugin="dsnp_arpes_raw")
    rheed_image.ri.rotate(-0.4)

    rheed_image.ri.apply_image_center(center_xs[i], center_ys[i])
    rheed_image.ri.beta = beta
    rheed_image.ri.alpha = alpha_vect[i]

    rheed_image.ri.screen_roi_width = 70
    rheed_image.ri.screen_roi_height = 60

    rheed_image = high_pass_filter(rheed_image, sigma=3.0, threshold=0.7)

    rheed_images.append(rheed_image)


plot_images(rheed_images, vmin=5, vmax=30)
plt.show()

In [ ]:
from xrheed.kinematics.lattice import Lattice

# Crate (r3xr3)R30 reconstruction lattice
si_111_r3 = Lattice.from_surface_hex(a=3.84*np.sqrt(3), label='r3')
si_111_r3.rotate(30)

In [ ]:
from xrheed.kinematics.ewald import Ewald

ew_list = []

for i in range(len(rheed_images)):
    ew = Ewald(si_111_r3, rheed_images[i])
    ew.calculate_ewald()
    ew_list.append(ew)

fig, axs = plt.subplots(3, 1, sharex=True, figsize=(8, 10))

for i in range(len(rheed_images)):
    ax = axs[i]
    ew = ew_list[i]

    ew.plot(ax=ax, show_image=True, vmin=5, vmax=30)

plt.show()

In [ ]:
from xrheed.conversion.image import transform_image_to_kxky

In [ ]:
trans_images = []
for rheed_image in rheed_images:
    trans_image = transform_image_to_kxky(rheed_image, rotate=True, point_symmetry=True)
    trans_images.append(trans_image)

Convert a list into a Dataset

In [ ]:
import xarray as xr

# Create a dataset from the list of DataArrays
trans_image_dataset = xr.concat(trans_images, dim='alpha_vect')

# Assign the alpha_vect values to the new dimension
trans_image_dataset = trans_image_dataset.assign_coords(alpha_vect=alpha_vect)

In [ ]:
fig, axs = plt.subplots(
    1, len(trans_image_dataset), figsize=(10, 4), sharex=True, sharey=True
)

for idx in range(len(trans_image_dataset)):
    ax = axs[idx]
    trans_image_dataset.isel(alpha_vect=idx).plot(
        ax=ax, cmap="gray", add_colorbar=False, vmin=10, vmax=30
    )

    si_111_r3.plot_reciprocal(ax=ax, facecolor='none', edgecolor='c', s=50)

    ax.set_facecolor("black")
    ax.set_xlim(-3, 3)
    ax.set_ylim(-3, 3)
    ax.set_aspect(1.0)
    
plt.tight_layout()
plt.show()

Mean the values along alpha dimension

In [ ]:
trans_image_mean = trans_image_dataset.mean(dim='alpha_vect')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))

trans_image_mean.plot(ax=ax, cmap='gray', add_colorbar=False, vmin=7, vmax=20)

si_111_r3.plot_reciprocal(ax=ax, facecolor='none', edgecolor='c', s=50)

ax.set_aspect(1)
ax.set_facecolor('black')
ax.set_xlim(-2.5, 2.5)
ax.set_ylim(-2.5, 2.5)

plt.show()